<a href="https://colab.research.google.com/github/anama-1104/cis677/blob/main/Lab_activity_Amdahl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab Activity - Amdahl's law

Amdahl's law (https://en.wikipedia.org/wiki/Amdahl%27s_law) provides important insight into the limits of succesful parallelization.

The key observation is that parts of the execution of a program are inherently sequential and cannot be parallelized.
So, if the parallelizable portion of the code is parallelized in a very efficient manner, the portion of the execution time that is spent on the sequential part cannot be made faster.

This lab activity is very similar to what you did for your first programming project. The problem, the minimum dominating set, is the same and the code is almost identical. The only difference is that the time required to read the file is now timed separately.

For the purposes of this lab activity, we will make two assumptions:



1.   The code that solves the problem can be parallelized
2.   The code that reads the input matrix cannot be parallelized

Run the programs and write down the times required to read the file and to solve the problem.

Use Amdahl's law to calculate the maximum speedup for each of the three test files.

Notice that the maximum speedup will be higher for the larger problems,
that is, the maximum speedup increases as the size of the problem increases.

This analysis can be useful to understand the kind of theoretical execution time improvements that can be obtained by parallelizing code, depending on the nature of the code and the size of the problem.



In [1]:
%%writefile original_python.py
import sys
import time
import numpy as np

def read_adjacency_matrix(file_name):
  file_object = open(file_name, "r")
  # Input the number of rows and columns
  size = int(file_object.readline())
  rows = size
  cols = size
  # Initialize an empty matrix
  matrix = []

  # Input the matrix elements
  for i in range(rows):
    row = list(map(int, file_object.readline().split()))
    matrix.append(row)
  return matrix,size

# Convert an integer into a set of nodes
def convert_from_int_to_set(integer,size):
  set_of_nodes = []
  mask = 1
  for i in range(size):
    if ((mask & integer) != 0):
      set_of_nodes.append(i)
    mask = mask * 2
  return set_of_nodes

# Find the maximum independent set
def find_max_ind_set(adj_mat_numpy,size):
  max_independent_set_size = 0
  max_independent_set = []

  size_of_power_set = 1
  for i in range(size):
    size_of_power_set *= 2
  # print("The power set has ",size_of_power_set," elements")
  array_with_sizes = np.zeros(size_of_power_set)
  for i in range(size_of_power_set):
    this_set = convert_from_int_to_set(i,size)
    is_independent = True
    for n1 in this_set:
      for n2 in this_set:
        if (adj_mat_numpy[n1][n2] == 1):
          is_independent = False
    if (is_independent):
      array_with_sizes[i] = len(this_set)
    else:
      array_with_sizes[i] = 0


  max_independent_set_size = np.max(array_with_sizes)
  max_independent_set = np.where(array_with_sizes == max_independent_set_size)[0]
  # print("The max independent sets are encoded by: ",max_independent_set)
  return max_independent_set_size



if __name__ == "__main__":
# Read the content of the file with the a passed in the command line
# that contain the matrices to be multiplied
  start_read_file = time.time()
  adj_matrix,size = read_adjacency_matrix(sys.argv[1])
  adj_matrix_numpy = np.array(adj_matrix)
  end_read_file = time.time()
  elapsed_read_file = end_read_file - start_read_file
  print("Time required to read the file in python: ",elapsed_read_file)
  start_time = time.time()
  max_independent_set_size = find_max_ind_set(adj_matrix_numpy,size)
  end_time = time.time()
  elapsed_time = end_time - start_time
  print("Time required to complete the computation in python: ",elapsed_time)
  # print("The size of the maximum independent set is: ",max_independent_set_size)

Writing original_python.py


In [2]:
%%writefile k16.txt
16
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Writing k16.txt


In [3]:
%%writefile k18.txt
18
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Writing k18.txt


In [4]:
%%writefile k20.txt
20
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Writing k20.txt


In [5]:

!python original_python.py k16.txt

Time required to read the file in python:  0.0003101825714111328
Time required to complete the computation in python:  2.750727653503418


In [6]:

!python original_python.py k18.txt

Time required to read the file in python:  0.0002918243408203125
Time required to complete the computation in python:  7.500609874725342


In [8]:

!python original_python.py k20.txt

Time required to read the file in python:  0.0003311634063720703
Time required to complete the computation in python:  39.47301888465881


**Calculating Maximum Speedup Using Amdahl's Law**


## Timing Results

### k16.txt
* Time to read file: 0.000310 seconds
* Time to compute:  seconds:  2.750
* Total time: 2.750310 seconds

### k18.txt
* Time to read file: 0.000291 seconds
* Time to compute: 7.500seconds
* Total time:7.500291 seconds

### k20.txt
* Time to read file: 0.000331 seconds
* Time to compute: 39.473 seconds
* Total time:39.473331 seconds

## Amdahl's Law Formula

**Formula:** Maximum Speedup = 1 / S  

**Where:** S = (Read Time) / (Total Time)



## Maximum Speedup for k16.txt

1. S = 0.000310 / 2.750310 = 0.0001127
2. Maximum Speedup = 1 / 0.0001127 =  8,872



## Maximum Speedup for k18.txt

1. S = 0.000291/ 7.500291 = 0.0000388
2. Maximum Speedup = 1 / 0.0000388 = 25,773


## Maximum Speedup for k20.txt

1. S = 0.000331 / 39.473331 = 0.00000839
2. Maximum Speedup = 1 / 0.00000839  =  119,144

The maximum speedup increases as the problem size increases:

k16 8,872

k18 25,773

k20 119,144


When looking at your results, one clear pattern stands out: the time it takes to read the file — the sequential part of the process — barely changes. It’s always around 0.0003 seconds, no matter how large the problem is. In contrast, the actual computation time, which can be parallelized, grows dramatically as the problem gets bigger: about 2.75 seconds for k16, 7.5 seconds for k18, and almost 40 seconds for k20.

This means that as the problems get larger, the fixed cost of reading the file becomes almost irrelevant compared to the heavy lifting of computation. For the smaller case (k16), reading the file still accounts for a tiny fraction of the work, but by the time we reach k20, it’s practically invisible in the total runtime.

This behavior reflects Amdahl’s Law perfectly: the sequential part doesn’t grow, but the parallelizable work does. So, the bigger the problem, the less the sequential overhead matters, and the more benefit we can expect from parallelization. In simple terms — small problems don’t show off the power of parallelism very well, but as the workload grows, parallel computing really begins to shine